NAME: ANMOL POONIA


ID: 202318009

PRGRAM: MSC DS

Task B:

This segment of the assignment is designed to enhance your understanding of the API
call process and dataset creation. Utilize any freely available API endpoint on Rapidapi to
generate a dataset.

In [23]:
import requests
import pandas as pd

urls = [
    "https://open-weather13.p.rapidapi.com/city/london",
    "https://open-weather13.p.rapidapi.com/city/Delhi",
    "https://open-weather13.p.rapidapi.com/city/Paris",
    "https://open-weather13.p.rapidapi.com/city/Moscow",
    "https://open-weather13.p.rapidapi.com/city/Cairo"
]

headers = {
    "X-RapidAPI-Key": "40685cca62msh04b502cab674d9cp160b4ejsned84208b63bc",
    "X-RapidAPI-Host": "open-weather13.p.rapidapi.com"
}

# Create an empty list to store individual DataFrames for each city
dfs = []

# Loop through each URL
for url in urls:
    response = requests.get(url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        try:
            # Try to parse the JSON response
            weather_data = response.json()

            # Extract relevant information from the API response
            dataset = {
                'City': weather_data.get('name', None),
                'Temperature': weather_data.get('main', {}).get('temp', None),
                'Humidity': weather_data.get('main', {}).get('humidity', None),
                'Description': weather_data.get('weather', [{}])[0].get('description', None),
                'Wind Speed': weather_data.get('wind', {}).get('speed', None),
            }

            # Create a Pandas DataFrame for each city
            df = pd.DataFrame([dataset])
            dfs.append(df)

        except Exception as e:
            print(f"Error parsing JSON for {url}: {e}")

    else:
        print(f"Request for {url} failed with status code: {response.status_code}")
        print(response.text)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(final_df)


     City  Temperature  Humidity                   Description  Wind Speed
0  London        49.84        80                 broken clouds       14.97
1   Delhi        57.20       100  thunderstorm with light rain        3.44
2   Paris        50.50        70              scattered clouds        9.22
3  Moscow        30.79        95               overcast clouds        5.88
4   Cairo        57.96        47                    light rain       14.97


 # Task C:

In this part of the assignment, you will learn how to construct a dataset by scraping a
website online. Utilize BeautifulSoup or any other suitable scraping library to extract all the
details of Amazon products and compile a dataset.

In [8]:
! pip3 install beautifulsoup4

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Function to extract Product Title

In [10]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [6]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [7]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,,4.6 out of 5 stars,"1,345 ratings",In Stock
1,PlayStation 4 500GB Console (Renewed),,4.1 out of 5 stars,947 ratings,Not Available
2,PlayStation 4 Slim 1TB Console - Black (Renewed),,4.2 out of 5 stars,"1,485 ratings",In Stock
3,Sony PlayStation 4 Slim Limited Edition 1TB Ga...,,4.3 out of 5 stars,"1,341 ratings",In Stock
4,"Sony PlayStation 4 Pro w/ Accessories, 1TB HDD...",,4.0 out of 5 stars,797 ratings,In Stock
5,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,338 ratings,In Stock
6,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,40 ratings,In Stock
7,Spider-Man: Miles Morales - For PlayStation 4,,4.6 out of 5 stars,184 ratings,Only 2 left in stock - order soon.
8,Hogwarts Legacy - PlayStation 4 | English | EU...,,4.7 out of 5 stars,545 ratings,In Stock
9,PlayStation 4 Slim 500GB Console [Discontinued...,,4.4 out of 5 stars,"1,360 ratings",In Stock


from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('rating').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
amazon_df.groupby('availability').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['availability'].value_counts()
    for x_label, grp in amazon_df.groupby('rating')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('rating')
_ = plt.ylabel('availability')